# 1. Import the necessary libraries for you to be able to query 4square API

In [10]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
import time

import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element

In [8]:
load_dotenv()

True

In [9]:
token = os.getenv("token")

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [53]:
# your code here

where = "Pamplona, 96, 08018 Barcelona"
url_geocode = f"https://geocode.xyz/{where}?json=1"
res = requests.get(url_geocode)
res.json()


{'standard': {'stnumber': '96',
  'addresst': 'Pamplona',
  'statename': 'Catalonia',
  'postal': '08018',
  'region': 'Catalonia',
  'prov': 'ES',
  'city': 'Barcelona',
  'countryname': 'Spain',
  'confidence': '0.9'},
 'longt': '2.19035',
 'alt': {},
 'elevation': {},
 'latt': '41.39780'}

In [18]:

url = "https://api.foursquare.com/v3/places/search?query=coffe%20place&ll=41.3977421%2C2.1902493&radius=500&categories=13032%2C13033%2C13034%2C13035%2C13036%2C13037&limit=10"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3zAFHV1WYjmUHowwITpbxqVLcnhXxxS9RdFUz7R54YqQ="
}

response = requests.get(url, headers=headers)

response.json()

{'results': [{'fsq_id': '530c5e6911d2be0b8d22e27a',
   'categories': [{'id': 13034,
     'name': 'Café',
     'short_name': 'Café',
     'plural_name': 'Cafés',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
      'suffix': '.png'}},
    {'id': 13035,
     'name': 'Coffee Shop',
     'short_name': 'Coffee Shop',
     'plural_name': 'Coffee Shops',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'}},
    {'id': 13065,
     'name': 'Restaurant',
     'short_name': 'Restaurant',
     'plural_name': 'Restaurants',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 135,
   'geocodes': {'main': {'latitude': 41.396937, 'longitude': 2.191519},
    'roof': {'latitude': 41.396937, 'longitude': 2.191519}},
   'link': '/v3/places/530c5e6911d2be0b8d22e27a',
   'location': {'address': 'Calle

# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [35]:
# your code here
def requests_for_foursquare (query, lat, lon, radius = 500, sort = "DISTANCE" ,limit = 1):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&sort={sort}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("te has pasado")

In [54]:
lat_ih = 41.39780
lon_ih = 2.19035

res = requests_for_foursquare ("coffe", lat_ih, lon_ih, radius = 500, sort = "DISTANCE" , limit = 5)
res

{'results': [{'fsq_id': '4bd94b1ccc5b9521e149f24f',
   'categories': [{'id': 13022,
     'name': 'Sports Bar',
     'short_name': 'Sports Bar',
     'plural_name': 'Sports Bars',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/sportsbar_',
      'suffix': '.png'}},
    {'id': 13049,
     'name': 'Diner',
     'short_name': 'Diner',
     'plural_name': 'Diners',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/diner_',
      'suffix': '.png'}},
    {'id': 13068,
     'name': 'American Restaurant',
     'short_name': 'American',
     'plural_name': 'American Restaurants',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 34,
   'geocodes': {'drop_off': {'latitude': 41.397718, 'longitude': 2.189932},
    'main': {'latitude': 41.397817, 'longitude': 2.190007},
    'roof': {'latitude': 41.397817, 'longitude': 2.190007}},

In [41]:
name = res["results"][0]["name"]
name

'Sopa de Lletres'

In [48]:
res["results"][0]["geocodes"]["main"]["longitude"]

2.158932

# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [74]:
def foursquare_places (venue, coordinates):
    
    lat, lon = str(coordinates).split(',')

    res = requests_for_foursquare (venue, lat, lon, radius = 500, sort = "DISTANCE" , limit = 10)
    venue = res["results"]  
    results = []

    for i in venue:
        name = i["name"]
        lon = i["geocodes"]["main"]["longitude"]
        lat = i["geocodes"]["main"]["latitude"]

        results.append( {
            "name": name,
            "lat": lat,
            "lon": lon
            })
 
    return results

In [80]:
coordinates= "41.39780,2.19035"
results = foursquare_places("restaurant",coordinates)

In [81]:
df = pd.DataFrame(results)
df

,name,lat,lon
0,Madrelievito Laboratorio,41.398909,2.192294
1,L'Estoneta,41.399145,2.188496
2,Espai Joliu,41.398735,2.195069
3,Restaurante la Fonda,41.401379,2.188801
4,Mian Dao,41.401337,2.188788
5,Granja Mabel,41.394816,2.187354
6,Van Van Var,41.394219,2.190291
7,Pasta Chef,41.400538,2.186726
8,Mannà,41.395016,2.194237
9,Eva-2,41.401636,2.189006


# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [82]:
# your code here
df.to_json("data/coffe_shops.jason")

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [ ]:
# your code here